# `where` are the `layerDefs`?

Currently there is no official api published on how to further filter the result list returned by `/identify`. There is a working implementation using the query parameter `where`, which is used internally by the viewer, and maybe also externally (to be clarified). This query parameter is however not documented. Upon two requests from clients there have been made attempts to improve and stabilize the possibility to filter. In the current WIP, two query params are used that use exactly the same parsing logic in the background, but different formats in the query, `where` and `layerDefs`. Ideally, we'd just making one of them publicly available (and documented). You find below on how to use them.

Note: Since `where` might sound too intriguing not to try any sqlinjection stuff, we could rename it to `filter`. That's what it actually does. 

In [87]:
import requests
import pprint
import urllib.parse

# BRANCH = 'mom_extend_find'
BRANCH = 'ltrea_extend_find'


def get_url(params):
    """ params needs to be a dict of the form
    where = {
        "branch":'',
        "paramName":'where',
        "paramValue":"plzo+=+'8302 Kloten'"
    }
    """
    _url = """https://mf-chsdi3.dev.bgdi.ch/
{branch}/
rest/services/all/MapServer/
identify?
geometry=2577070,1184860
&mapExtent=0,0,100,100
&imageDisplay=100,100,100
&geometryFormat=geojson
&geometryType=esriGeometryPoint
&lang=de
&layers=all:{layer}
&returnGeometry=true
&tolerance=500000
&sr=2056
&{paramName}={paramValue}
    """
    url = _url.format(**params).strip().replace('\n','')
    print("\n=================\nrequesting {}".format(url))
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json().get('results', [])
        print("--> success! {} results".format(len(results)))
        pprint.pprint(next(iter(results), []))
    else:
        print("--> example :{}: returned error code {}: {}".format(example, response.status_code, response.text))

def encode_param(query):
    return urllib.parse.quote(query)

{"ch.swisstopo.amtliches-strassenverzeichnis":"plzo%20ilike%20'%Basel%'","ch.swisstopo.amtliches-strassenverzeichnis":"label%20ilike%20'%Hauptstrasse%'"}

examples = {
    "ch.swisstopo.amtliches-strassenverzeichnis": [
        "plzo = '8302 Kloten'",
        "plzo ilike '8302%' or 1=1",
        "blah ilike '8302%'",
        "gdename ilike 'sel%'",
        "label ilike '%Hauptstrasse%'",
    ],
    "ch.bfe.ladestellen-elektromobilitaet": [
        "IsOpen24Hours = true", 
        "QueryAuthenticationModes ilike '%nfc%'", 
        "Longitude > 7.476"        
    ]
}


In [91]:
sqlinj = {
    "ch.swisstopo.amtliches-strassenverzeichnis": ["1=1"]
}
for layer, layer_examples in sqlinj.items():
    for example in layer_examples:
        where_params = {
            "branch":BRANCH,
            "paramName":'where',
            "layer":layer,
            "paramValue":encode_param(example)
        }
        get_url(where_params)


requesting https://mf-chsdi3.dev.bgdi.ch/ltrea_extend_find/rest/services/all/MapServer/identify?geometry=2577070,1184860&mapExtent=0,0,100,100&imageDisplay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.swisstopo.amtliches-strassenverzeichnis&returnGeometry=true&tolerance=500000&sr=2056&where=1%3D1
--> example :1=1: returned error code 400: {"status":"error","code":400,"detail":"The where/layerDefs clause is not valid for ch.swisstopo.amtliches-strassenverzeichnis."}


## where
When using `where` as query param, the filter expression with the attribute(s) contained in the layer specified for identify are directly passed, e.g.
```
&where=plzo+=+'8302 Kloten'
```
**Note: The filter expression must be correctly url-encoded!**

### Examples

In [86]:
for layer, layer_examples in examples.items():
    for example in layer_examples:
        where_params = {
            "branch":BRANCH,
            "paramName":'where',
            "layer":layer,
            "paramValue":encode_param(example)
        }
        get_url(where_params)


requesting https://mf-chsdi3.dev.bgdi.ch/ltrea_extend_find/rest/services/all/MapServer/identify?geometry=2577070,1184860&mapExtent=0,0,100,100&imageDisplay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.swisstopo.amtliches-strassenverzeichnis&returnGeometry=true&tolerance=500000&sr=2056&where=plzo%20%3D%20%278302%20Kloten%27
--> success! 201 results
{'bbox': [2686780.9, 1256424.6, 2687007.5, 1256716.0],
 'featureId': 5419400,
 'geometry': {'coordinates': [[[2687007.5, 1256424.6], [2686862.1, 1256511.9]],
                              [[2686862.1, 1256511.9],
                               [2686900.2, 1256585.6],
                               [2686918.9, 1256611.8]],
                              [[2686862.1, 1256511.9],
                               [2686829.1, 1256530.4],
                               [2686786.2, 1256551.1],
                               [2686781.1, 1256556.9],
                               [2686780.9, 1256562.7],
       

## layerDefs
When using the query parameter `layerDefs`, the layer on which the filter should be applied is again specified but must be the same as specified in the `layer` parameter for identify, e.g.
```
{"ch.swisstopo.amtliches-strassenverzeichnis": "plzo = '8302 Kloten'"}
```
**Note: also here, the whole expression above must be correctly url encoded!**

### Examples

In [84]:
import json

for layer, layer_examples in examples.items():
    for example in layer_examples:
        layerDefsDict = {layer:example}
        layerDefs_params = {
            "branch":BRANCH,
            "layer":layer,
            "paramName":'layerDefs',
            "paramValue":encode_param(json.dumps(layerDefsDict))
        }
        get_url(layerDefs_params)


requesting https://mf-chsdi3.dev.bgdi.ch/ltrea_extend_find/rest/services/all/MapServer/identify?geometry=2577070,1184860&mapExtent=0,0,100,100&imageDisplay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.swisstopo.amtliches-strassenverzeichnis&returnGeometry=true&tolerance=500000&sr=2056&layerDefs=%7B%22ch.swisstopo.amtliches-strassenverzeichnis%22%3A%20%22plzo%20%3D%20%278302%20Kloten%27%22%7D
--> success! 201 results
{'bbox': [2686780.9, 1256424.6, 2687007.5, 1256716.0],
 'featureId': 5419400,
 'geometry': {'coordinates': [[[2687007.5, 1256424.6], [2686862.1, 1256511.9]],
                              [[2686862.1, 1256511.9],
                               [2686900.2, 1256585.6],
                               [2686918.9, 1256611.8]],
                              [[2686862.1, 1256511.9],
                               [2686829.1, 1256530.4],
                               [2686786.2, 1256551.1],
                               [2686781.1, 1256

### Open questions

- cross-filter functionality in ESRI??lay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.swisstopo.amtliches-strassenverzeichnis&returnGeometry=true&tolerance=500000&sr=2056&layerDefs={%22ch.swisstopo.amtliches-strassenverzeichnis%22:%20%22plzo%20=%20%278302%20Kloten%27%20AND%20plzo%20ilike%20%273000%%27%22}

- who is using where other than frontend?
- how to configure in bod to identify layer that is visualized as json but queryable?